## Intro

This is inspired by 
Article (likas2001probability) Likas, A. Probability density estimation using artificial neural networks Computer physics communications, Elsevier, 2001, 135, 167-175

But rather than estimating the working with a network, we will instead work with its derivitive.
This will let us replace their integration with a derivative.

Note that this method only works for compact supports



They use the PDF is given by $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p) dz}$$
and in their case $h=N(x,p)$  a neural network with weight and bias parameters $p$.
Where $S$ is a compact support. (That means bounded)


But if instead we say $h=\frac{\partial N(x,p)}{\partial x}$,

then $$p_h(x,p) = \dfrac{h(x,p)}{\int_S h(z,p)}=\dfrac{\frac{\partial N(x,p)}{\partial x}}{N(max(S),p) - N(min(S), p)}$$

The denominator is ofcourse more complex for non-1D values of S.


The loss function given is the negative log-likelihood of the set of training samples $X$
$$L(p) = -\sum_{\forall x \in X} ln(h(x,p))  + |X| ln(\int_S h(z,p) dx)$$

Which befomes:

$$L(p) = -\sum_{\forall x \in X} log(\frac{\partial N(x,p)}{\partial x})  + |X|(ln(N(max(S),p)-N(min(S),p)) dx$$

In [1]:
using DensityEstimationML
function only(itr)
    state = start(itr)
    val,state = next(itr, state)
    @assert(done(itr,state))
    return val
end

only (generic function with 1 method)

In [2]:
using TensorFlow
using Distributions
using StatsBase
using StaticArrays

In [3]:
dataset = GenerateDatasets.Likas1()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(dataset))

2017-09-21 16:34:05.339908: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.339940: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.339945: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
2017-09-21 16:34:05.563312: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:893] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2017-09-21 16:34:05.563927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with propert

pdf = <Tensor pdf:1 shape=unknown dtype=Float64>


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2cc7fddc30), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [4]:
pdf(est, data)

5000-element Array{Float64,1}:
 0.0416668
 0.0416668
 0.0416668
 0.0416665
 0.0416668
 0.0416668
 0.0416668
 0.0416668
 0.0416665
 0.0416665
 0.0416668
 0.0416665
 0.0416666
 ⋮        
 0.0416668
 0.0416667
 0.0416668
 0.0416668
 0.0416667
 0.0416668
 0.0416668
 0.0416666
 0.0416668
 0.0416665
 0.0416668
 0.0416667

In [5]:
liklpdf(est, data)

LoadError: [91mUndefVarError: liklpdf not defined[39m

In [5]:
@which pdf(est, DensityEstimationML.autostatic_array(data))

pdf(est::DensityEstimationML.NeuralDensityEstimator, ts::AbstractArray) in DensityEstimationML at /home/uniwa/students2/students/20361362/linux/.julia/v0.6/DensityEstimationML/src/estimator.jl:33

In [6]:
DensityEstimationML.autostatic_array(data) |> typeof <: StaticVector{5000}

true

In [7]:
StaticVector{5000} <: AbstractVector

true

In [22]:
function Distributions.pdf(est::NeuralDensityEstimator{1}, ts::StaticArrays.StaticArray{Tuple{1},Float64,1})
    println("hi")
end

In [19]:
DensityEstimationML.autostatic_array(data) |> typeof |> supertype

StaticArrays.StaticArray{Tuple{5000},Float64,1}

In [25]:
pdf(est, data)

LoadError: [91mUndefVarError: _pdf not defined[39m

In [ ]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(dataset))
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [15]:
expand(:(x[end]))

:(getindex(x, (Base.endof)(x)))

In [10]:
gradients(gr["yt"], gr["t"])[1,:] |> get_shape

LoadError: [91mUndefVarError: gr not defined[39m

In [17]:
reshape(RowVector([1,2,3]), Val(2))

1×3 RowVector{Int64,Array{Int64,1}}:
 1  2  3

In [ ]:
println.(sort(gr |> keys |> collect))

In [6]:
using Plots
using IJulia

"""
Function returning a function that will display a running plot.
WARNING: Introducting or removing any variables is not supported.
And will silently error.
"""
function running_plot()
    epochs = Int[]
    record = Dict()
    function inner(epoch, vars::Associative)
        for (var, values) in vars
            value = only(values) #Incase it was an array
            past = get!(record, var) do
                typeof(value)[]
            end
            push!(past, value)
        end
        push!(epochs, epoch)
        
        IJulia.clear_output(true)
        plot(epochs, hcat(values(record)...); label=hcat(keys(vars)...), layout=length(vars)) |> IJulia.display       
    end
end


running_plot

In [11]:
function demonstration_plot(est, dataset, data, args...; kwargs...)
    X = minimum(approximate_support(dataset)) : 0.01 : maximum(approximate_support(dataset)) |> collect
    println("True loglikelihood      = $(loglikelihood(dataset, data))")
    println("Estimated loglikelihood = $(loglikelihood(est, data))")
    plot([X], [pdf(est,X), data],
        #xlims= approximate_support(dataset),
        xlims= (first(X), last(X)),
        seriestype = [:path :histogram],
        layout=(2,1),
        legend=false,
        nbins=[1  length(data)÷10],
        args...; kwargs...
    )
end

demonstration_plot (generic function with 1 method)

In [12]:
function demo(dataset, layers, epochs=20_000; max_conditioning_epochs=2000)
    data = original_sample(dataset)
    @show loglikelihood(dataset, data)
    est = NeuralDensityEstimator(layers, approximate_support(dataset))

    condition!(est; max_epochs = max_conditioning_epochs)
    println("Conditioning Done")
    Plots.gr()
    fit!(est, data; epochs=epochs, callback=running_plot())
    println("Fitting Done")
    plotly()
    demonstration_plot(est, dataset, data) |> IJulia.display
    
    est    
end

demo (generic function with 2 methods)

In [13]:
est=demo(GenerateDatasets.Likas1(), [64, 64, 256], 1_000)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 ysmax 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 20000 
 
 
 30000 
 
 
 
 
 
 
 working_loss 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 -30000 
 
 
 -20000 
 
 
 
 
 
 
 loglikelihood 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 ysmin

Fitting Done
True loglikelihood      = -10471.934928226601
Estimated loglikelihood = -14553.320976478046


DensityEstimationML.NeuralDensityEstimator{1}(Session(Ptr{Void} @0x00007f2c98586af0), <Tensor Group:1 shape=unknown dtype=Any>, <Tensor Group_2:1 shape=unknown dtype=Any>, <Tensor t:1 shape=(1, ?) dtype=Float32>, <Tensor pdf:1 shape=unknown dtype=Float64>)

In [ ]:
let 
    sess = Session(Graph())
    X=constant([1.0,2,3])
    y=Ops.mul(X,X)
    @show y
end

In [ ]:
loglikelihood(GenerateDatasets.Likas1(), [2.0])

In [ ]:
GenerateDatasets.Likas1()

In [ ]:
sort(component_weights(est))

In [ ]:
demo(GenerateDatasets.Likas2(), [64,64], 20_000)

In [ ]:
demo(GenerateDatasets.MagdonIsmailAndAtiya(), [32], 10_000)

In [ ]:
demo(Arcsine(1,4), [64,64], 20_000)

In [ ]:
dataset = GenerateDatasets.Likas3()
data = original_sample(dataset)
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

In [ ]:
@show loglikelihood(dataset, data)

In [ ]:
est = NeuralDensityEstimator([64], approximate_support(GenerateDatasets.Likas3()))

In [ ]:
condition!(est)

In [ ]:

fit!(est, data'; epochs=10_000, callback=running_plot())

In [ ]:

fit!(est, data; epochs=10_000, callback=running_plot())
println("Fitting Done")

In [ ]:
sess = est.sess
run(sess, est.pdf, Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [ ]:
sess = est.sess
run(sess, sess.graph["numerator"], Dict(est.t=>[0.1 0.1; 0.1 0.1; 0.3 0.6]'))

In [ ]:
data

In [ ]:
run(sess, gather(constant(data), 1))

In [ ]:
plotly()

In [ ]:
"""
    meshgrid(xs, ys, zfun)

Evalates `zfun(x,y)` at each point in `xs`, and `ys`.
Returns 3 vectors, a list of x points a list of y points and the value of z at that point.
"""
function meshgrid(xs, ys, zfun) 
    # There is a cute generalisation of this with a `@generated` function
    xpoints = eltype(xs)[]
    ypoints = eltype(ys)[]
    zpoints = typeof(zfun(xs[1],ys[1]))[]
    sizehint!.([xpoints, ypoints, zpoints], length(xs)*length(ys))
    for x in xs, y in ys
        push!(xpoints,x)
        push!(ypoints,y)
        push!(zpoints,zfun(x,y))
    end
    
    xpoints, ypoints, zpoints
end


In [ ]:
X=-0.1:0.01:0.3
Y=-0.1:0.01:0.3
scatter3d(meshgrid(X,Y, (x,y)->pdf(est, [x,y]))...)

In [ ]:
pdf(est, SMatrix{1,2}([0.1 0.1]))

In [ ]:
est

In [ ]:
size([0.1 0.1])

In [ ]:
?SArray